<a href="https://colab.research.google.com/github/Jesus2421/Hamilton/blob/main/Hamilton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install torch
!pip install sounddevice scipy
!pip install SpeechRecognition pyttsx3
!pip install vllm

In [11]:
from transformers import pipeline

asistente = pipeline("text-generation", model="microsoft/DialoGPT-medium", batch_size=1)

print("Hamilton: Hola señor, ¿en qué puedo ayudarte?")
while True:
    entrada = input("usuario: ")
    if entrada.lower() in ["salir", "terminar"]:
        print("Hamilton: Hasta luego.")
        break
    respuesta = asistente(entrada, max_new_tokens=200)
    print(f"Hamilton: {respuesta[0]['generated_text']}")


Device set to use cpu


Hamilton: Hola señor, ¿en qué puedo ayudarte?
usuario: Hola Hamilton, Bienvenido a la vida


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hamilton: Hola Hamilton, Bienvenido a la vida.
usuario: Hamilton, ¿estas ahi?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hamilton: Hamilton, ¿estas ahi?
usuario: terminar 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hamilton: terminar izvodno
usuario: salir
Hamilton: Hasta luego.


In [12]:
import os

# Verifica si el archivo existe
if os.path.exists("train.json"):
    print("✅ Archivo train.json cargado correctamente.")
else:
    print("❌ Archivo no encontrado.")


✅ Archivo train.json cargado correctamente.


In [38]:
pip install wandb

In [44]:
import os
os.environ["WANDB_DISABLED"] = "true"
import json
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import wandb


# Login to wandb
wandb.login()

if os.path.exists("train.json"):
    print("✅ Archivo train.json cargado correctamente.")
    # Cargar el modelo y tokenizador
    modelo_base = "microsoft/DialoGPT-small"
    tokenizer = AutoTokenizer.from_pretrained(modelo_base)
    # Asegurarse de que el tokenizador tenga un token de padding
    tokenizer.pad_token = tokenizer.eos_token  # Usamos el token EOS como pad_token
    # Continuamos con el resto del código
    def tokenize(batch):
      inputs = [
          prompt + tokenizer.eos_token + response + tokenizer.eos_token
          for prompt, response in zip(batch["prompt"], batch["response"])
          ]
      encodings = tokenizer(
          inputs,
          truncation=True,
          padding="max_length",
          max_length=128
          )
      # Agregamos las etiquetas como copia de input_ids
      encodings["labels"] = encodings["input_ids"].copy()
      return encodings
    # Tokenizar el dataset
    tokenized_dataset = dataset.map(tokenize, batched=True)
    # ✅ Configurar entrenamiento
    training_args = TrainingArguments(
        output_dir="./hamiton-model",
        per_device_train_batch_size=2,
        num_train_epochs=5,
        logging_dir="./logs",
        save_strategy="epoch",
         report_to="wandb"
    )

    training_args = TrainingArguments(
        output_dir="./hamiton-model",
        per_device_train_batch_size=2,
        num_train_epochs=5,
        logging_dir="./logs",
        save_strategy="epoch",
        report_to="none"
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
    )

    # ✅ Entrenar el modelo
    trainer.train()
else:
    print("❌ Archivo no encontrado.")



✅ Archivo train.json cargado correctamente.


Map:   0%|          | 0/17 [00:00<?, ? examples/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


In [45]:
model.save_pretrained("hamiton-v1")
tokenizer.save_pretrained("hamiton-v1")

('hamiton-v1/tokenizer_config.json',
 'hamiton-v1/special_tokens_map.json',
 'hamiton-v1/vocab.json',
 'hamiton-v1/merges.txt',
 'hamiton-v1/added_tokens.json',
 'hamiton-v1/tokenizer.json')

In [48]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 1. Load the trained model and tokenizer using the correct path
model_path = "hamiton-v1"  # Replace with the actual path if different
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# 2. Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 3. Update the responder function
def responder(pregunta, max_length=100):
    # Tokenize the input and add eos_token
    inputs = tokenizer.encode(pregunta + tokenizer.eos_token, return_tensors="pt").to(device)

    # Generate response
    output = model.generate(
        inputs,
        max_length=max_length,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_k=50,  # Experiment with these values
        top_p=0.95,  # Experiment with these values
        temperature=0.7,  # Experiment with these values
    )

    # Decode the generated response (omit the original input)
    respuesta = tokenizer.decode(output[:, inputs.shape[-1]:][0], skip_special_tokens=True)
    return respuesta

# 4. Interact with the model
while True:
    entrada = input("Tú: ")
    if entrada.lower() in ["salir", "exit"]:
        print("Hamilton: Hasta luego, señor.")
        break
    salida = responder(entrada)
    print("Hamilton:", salida)

Tú: ¿Cómo va el estado del sistema?
Hamilton: 


KeyboardInterrupt: Interrupted by user